In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão.

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [3]:
# Baixa a base de dados.
housing = fetch_california_housing()

# Divide o conjunto total de exemplos em conjuntos de treinamento e teste.
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

# Divide o conjunto de treinamento em conjuntos de treinamento (menor) e validação.
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# Aplica padronização às matrizes de atributos.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Criando e compilando o modelo.

+ Neste exemplo usaremos a API sequencial, mas as `callbacks` também funcionam com a API funcional.

In [4]:
model = keras.models.Sequential(
    [
        keras.layers.Input(shape=[8]),
        keras.layers.Dense(30, activation="relu", name='hidden1'),
        keras.layers.Dense(30, activation="relu", name='hidden2'),
        keras.layers.Dense(1, name='output')
    ]
)

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

### Criando a callback `ModelCheckpoint`.

+ A callback `ModelCheckpoint` **salva pontos de verificação do modelo em intervalos regulares** durante o treinamento, por **padrão ao final de cada época**.
    + O parâmetro `save_freq`, pode ser configurado com a string `'epoch'` ou um valor inteiro. Ao usar `'epoch'`, a callback salva o modelo após cada época.
    + Ao usar um valor inteiro, a callback salva o modelo ao final do número de mini-batches especificado.


+ Se usarmos um **conjunto de validação** durante o treinamento, podemos definir o parâmetro `save_best_only=True` ao criar a callback `ModelCheckpoint`.
    + Nesse caso, a callback só salvará o modelo quando seu desempenho no conjunto de validação for o melhor até o momento.


+ Dessa forma, não precisamos nos preocupar em treinar por muito tempo e sobreajustar ao conjunto de treinamento: basta restaurar o último modelo salvo após o treinamento e este será o melhor modelo no conjunto de validação.
    + Precisamos restaurar o modelo manualmente após o treinamento.


+ Esta é uma maneira simples de implementar a parada antecipada (i.e., *early-stop*).
    + Entretanto, o treinamento não é encerrado antecipadamente, ele ocorre até a última época definida.

In [5]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./my_keras_model.keras", save_best_only=True)

### Treinando e avaliando o modelo salvo.

In [6]:
# Treina o modelo com a callback especificada.
history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb]
                   )

# Carrega o melhor modelo no conjunto de validação.
model = keras.models.load_model("./my_keras_model.keras")

# Avalia o modelo.
mse_test = model.evaluate(X_test, y_test)

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 3.8979 - val_loss: 3.1561
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.8131 - val_loss: 1.1498
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.6668 - val_loss: 0.6078
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6140 - val_loss: 0.5340
Epoch 5/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.5795 - val_loss: 0.5076
Epoch 6/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.5526 - val_loss: 0.4884
Epoch 7/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.5307 - val_loss: 0.4714
Epoch 8/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5124 - val_loss: 0.4572
Epoch 9/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.4967 - val_loss: 0.4444
Epoch 10/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.4834 - val_loss: 0.4326
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4486


### Criando a callback `EarlyStopping`.

+ Outra maneira de implementar a **parada antecipada** é simplesmente usar a callback `EarlyStopping`.


+ **Ela interromperá o treinamento** quando não medir nenhum progresso no conjunto de validação por várias épocas (definidas pelo parâmetro `patience`) e, **opcionalmente, carrega o melhor modelo ao final do treinamento**.
    + Por padrão, a métrica avaliada para encerrar o treinamento é a perda no conjunto de validação.


+ Podemos combinar as duas callbacks para salvar os pontos de verificação do modelo com a `ModelCheckpoint` (no caso de seu computador travar) e interromper o treinamento mais cedo quando não houver mais progresso, com a `EarlyStopping` (para evitar desperdício de tempo e recursos).

In [7]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

### Treinando e avaliando o modelo.

+ O número de épocas pode ser definido como um **valor grande**, **pois o treinamento será interrompido automaticamente quando não houver mais progresso**.


+ Além disso, não há necessidade de restaurar o melhor modelo neste caso, pois a callback `EarlyStopping` armazenará os melhores pesos (que resultaram na menor perda de validação) e os restaurará ao final do treinamento quando `restore_best_weights=True`.

In [8]:
# Treina o modelo com as callbacks definidas.
history = model.fit(X_train, y_train,
                    epochs=200,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb]
                   )

Epoch 1/200
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.4719 - val_loss: 0.4229
Epoch 2/200
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4619 - val_loss: 0.4143
Epoch 3/200
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4532 - val_loss: 0.4069
Epoch 4/200
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4456 - val_loss: 0.4005
Epoch 5/200
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4390 - val_loss: 0.3947
Epoch 6/200
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4331 - val_loss: 0.3898
Epoch 7/200
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4279 - val_loss: 0.3855
Epoch 8/200
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4232 - val_loss: 0.3816
Epoch 9/200
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4190 - val_loss: 0.3781
Epoch 10/200
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4151 - val_loss: 0.3747
Epoch 11/200
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4115 - val_loss: 0.3718
Epoch 12/200
363/363 ━━━━━━━━━━━━━━━━━━━━

In [9]:
# O melhor modelo é carregado ao final.
mse_test = model.evaluate(X_test, y_test)

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3194


In [10]:
X_new = X_test[:3]
y_pred = model.predict(X_new)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


In [14]:
for i in range(len(y_pred)):
    print('Actual: %1.3f - Predicted: %1.3f' % (y_test[i], y_pred[i,0]))

Actual: 0.477 - Predicted: 0.569
Actual: 0.458 - Predicted: 1.428
Actual: 5.000 - Predicted: 4.539


### Criando nossa própria callback.

+ Se precisarmos de controle extra, podemos criar facilmente nossas próprias callbacks.


+ Por exemplo, a seguinte callback exibirá a **época** e a **proporção entre a perda de validação e a perda de treinamento** durante o treinamento **ao final de cada época** (por exemplo, para detectarmos overfitting).


+ Notem que a classe `PrintValTrainRatioCallback` herda da classe `keras.callbacks.Callback`.


+ Para imprimir a razão entre a perda de validação e a perda de treinamento **ao final de cada época** devemos sobrescrever o método `on_epoch_end` da classe `Callback`.
    + **OBS**.: para que a callback funcione, não podemos alterar a assinatura do método.
    
    
+ O parâmetro `epoch` recebe o número da época corrente e o parâmetro `logs` é um dicionário com informações que incluem as perdas de treinamento e validação.

In [15]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nMinha callback -> epoch: {:d} - val/train: {:.4f}".format(epoch, logs["val_loss"] / logs["loss"]))

### Treinando e avaliando o modelo.

In [16]:
# Instancia objeto da classe PrintValTrainRatioCallback.
val_train_ratio_cb = PrintValTrainRatioCallback()

# Treina o modelo com as callbacks definidas.
history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[val_train_ratio_cb]
                   )

# O melhor modelo é carregado ao final.
mse_test = model.evaluate(X_test, y_test)

Epoch 1/10
341/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3206
Minha callback -> epoch: 0 - val/train: 1.0343
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3199 - val_loss: 0.3210
Epoch 2/10
330/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3208
Minha callback -> epoch: 1 - val/train: 0.9759
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3197 - val_loss: 0.3026
Epoch 3/10
328/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3207
Minha callback -> epoch: 2 - val/train: 1.0184
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3195 - val_loss: 0.3156
Epoch 4/10
333/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3203
Minha callback -> epoch: 3 - val/train: 0.9878
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3193 - val_loss: 0.3059
Epoch 5/10
336/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3200
Minha callback -> epoch: 4 - val/train: 1.0443
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3190 - val_loss: 0.3233
Epoch 6/10
325/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - l

### Importante

+ Podemos implementar callbacks para cada um dos seguintes eventos:
    + on_train_begin(),
    + on_train_end(),
    + on_epoch_begin(),
    + on_epoch_begin(),
    + on_batch_end(),
    + on_batch_end().


+ Além disso, caso necessário, as callbacks também podem ser usadas durante a **avaliação** e a **predição**, por exemplo, para depuração. Para isso, devemos implementar alguns métodos.
    + Os métodos abaixo são chamados pelo método **evaluate()**:
        + on_test_begin(),
        + on_test_end(),
        + on_test_batch_begin(),
        + on_test_batch_end().
    + Os métodos abaixo são chamados pelo método **predict()**:
        + on_predict_begin(),
        + on_predict_end(),
        + on_predict_batch_begin(),
        + on_predict_batch_end().